In [1]:
import os
import numpy as np
import pandas as pd
import pickle 
import quandl

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)


ModuleNotFoundError: No module named 'quandl'

In [ ]:
quandl.ApiConfig.api_key = 'zZzhwTNJ1Z8SxApaAd8K'

In [ ]:
def get_quandl_data(quandl_code):
    cache_path = '{}.pkl'.format(quandl_code).replace('/', '-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)
        print('Loaded {} from cache'.format(quandl_code))
    except (OSError, IOError) as e:
        df = quandl.get(quandl_code, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_code, cache_path))
    return df

In [ ]:
btc_usd_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

In [ ]:
btc_usd_kraken.head()

In [ ]:
btc_trace = go.Scatter(x=btc_usd_kraken.index, y=btc_usd_kraken['Weighted Price'])
py.iplot([btc_trace])

In [ ]:
exchanges = ['COINBASE', 'BITSTAMP', 'ITBIT']

exchange_data = {}
exchange_data['KRAKEN'] = btc_usd_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

In [ ]:
def merge_dfs(dataframes, labels, col):
    series_dict = {}
    
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
    return pd.DataFrame(series_dict)

In [ ]:
btc_usd_df = merge_dfs(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [ ]:
btc_usd_df.tail()

In [ ]:
layout = go.Layout(
    title = 'Bitcoin Price By Exchange (USD)',
    legend = {'orientation': 'h'},
    xaxis = {'type': 'date'},
    yaxis = {'title': 'Price in USD'}
)

trace_arr = []
labels = list(btc_usd_df)
for index, label in enumerate(labels):
    series = btc_usd_df[label]
    trace = go.Scatter(x=series.index, y=series, name=label)
    trace_arr.append(trace)
    
fig = go.Figure(data=trace_arr, layout=layout)
py.iplot(fig)

In [ ]:
btc_usd_df['avg_usd_price'] = btc_usd_df.mean(axis=1)
btc_trace = go.Scatter(x=btc_usd_df.index, y=btc_usd_df['avg_usd_price'])
py.iplot([btc_trace])